In [ ]:
import numpy as np
import sys
import warnings
import time
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
# --- COMPETITORS ---
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
try:
    from xgboost import XGBClassifier
    from lightgbm import LGBMClassifier
except ImportError:
    print(">>> [WARNING] XGBoost or LightGBM not installed. Benchmarks will be limited.")

# --- GPU ACTIVATION ---
try:
    import cupy as cp
    print(">>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.")
    cp.random.seed(42)
    GPU_AVAILABLE = True
except ImportError:
    print(">>> [FATAL] GPU REQUIRED FOR TITAN.")
    sys.exit()

warnings.filterwarnings("ignore")

# ==========================================
# 1. THE MEMORY UNIT (The Synapse)
# ==========================================
class MEMORY_UNIT:
    def __init__(self, n_neurons, input_dim, seed, activation_func):
        cp.random.seed(seed)
        self.n_neurons = n_neurons
        self.activation = activation_func # <--- The Chosen One

        self.W_in = cp.random.uniform(-1.0, 1.0, (n_neurons, input_dim)).astype(cp.float32)
        self.W_cortex = cp.random.normal(0, 1.0 / np.sqrt(n_neurons), (n_neurons, n_neurons)).astype(cp.float32)

        # Spectral stabilizer (Simplified for universal AFs)
        try:
            v = cp.random.randn(n_neurons)
            for _ in range(5):
                v = self.W_cortex @ v
                v = v / cp.linalg.norm(v)
            max_eig = cp.linalg.norm(self.W_cortex @ v)
            if max_eig > 0: self.W_cortex /= max_eig
        except: pass
        self.readout = None

    def process(self, X_gpu):
        state = cp.zeros((X_gpu.shape[0], self.n_neurons), dtype=cp.float32)
        for _ in range(5):
            pre = (X_gpu @ self.W_in.T) + (state @ self.W_cortex.T)
            state = self.activation(pre) # <--- Use the scanned function
        return state

    # fit_residual and predict_correction remain the same
    def fit_residual(self, X_gpu, Residual_gpu, alpha=10.0):
        H = self.process(X_gpu)
        I = cp.eye(self.n_neurons, dtype=cp.float32)
        self.readout = cp.linalg.solve(H.T @ H + alpha*I, H.T @ Residual_gpu)

    def predict_correction(self, X_gpu):
        H = self.process(X_gpu)
        return H @ self.readout


# ==========================================
# 0. THE RESONANCE SCANNER (God's Eye)
# ==========================================
class RESONANCE_SCANNER:
    def __init__(self):
        self.activations = {}
        self._generate_100_functions()

    def _generate_100_functions(self):
        # 1. Standard Activations
        self.activations['tanh'] = lambda x: cp.tanh(x)
        self.activations['relu'] = lambda x: cp.maximum(0, x)
        self.activations['sigmoid'] = lambda x: 1 / (1 + cp.exp(-x))
        self.activations['swish'] = lambda x: x * (1 / (1 + cp.exp(-x)))
        self.activations['mish'] = lambda x: x * cp.tanh(cp.log(1 + cp.exp(x)))

        # 2. Harmonic Series (The "Siren" Frequencies)
        # We test different frequencies to find the "Pitch" of the data
        for freq in [0.5, 1.0, 1.5, 2.0, 3.0, 5.0, 8.0, 13.0, 21.0, 34.0, 55.0]: # Fibonacci Frequencies
            self.activations[f'sin_omega_{freq}'] = lambda x, f=freq: cp.sin(f * x)
            self.activations[f'cos_omega_{freq}'] = lambda x, f=freq: cp.cos(f * x)

        # 3. Gaussian Kernels (For Localized Spikes)
        for sigma in [0.1, 0.5, 1.0, 2.0, 5.0]:
            self.activations[f'gaussian_{sigma}'] = lambda x, s=sigma: cp.exp(-cp.square(x) / (2 * s**2))

        # 4. Hybrid & Exotic (The "Chaos" Functions)
        self.activations['sinc'] = lambda x: cp.sin(x) / (x + 1e-6)
        self.activations['morlet'] = lambda x: cp.cos(5*x) * cp.exp(-x**2/2) # Wavelet
        self.activations['x_sin_x'] = lambda x: x * cp.sin(x)
        self.activations['sin_tanh'] = lambda x: cp.sin(cp.tanh(x))

        # ... (We can generate more parametrically to reach 100+)
        # Generating fine-tuned frequency bands to hit 100+ variations
        for i in range(1, 60):
            f = 0.1 * i
            self.activations[f'fine_sin_{f:.1f}'] = lambda x, freq=f: cp.sin(freq * x)

    def scan(self, X_gpu, Residual_gpu, n_neurons=100):
        """
        Rapidly tests all activations on the FULL dataset to find the best fit.
        """
        best_name = "tanh"
        best_score = -np.inf

        # Create a single random projection to test the AFs fairly
        # We use a small probe size (n_neurons=100) for extreme speed
        cp.random.seed(42)
        W_probe = cp.random.uniform(-1.0, 1.0, (n_neurons, X_gpu.shape[1])).astype(cp.float32)

        # Pre-compute projection
        Projected_X = X_gpu @ W_probe.T

        print(f"   [SCANNER] Testing {len(self.activations)} Quantum Activation Functions...", end="")

        start = time.time()

        # Fast Loop
        for name, func in self.activations.items():
            try:
                # 1. Apply Candidate Activation
                H = func(Projected_X)

                # 2. Fast Ridge Solver (One-Shot Learning)
                # We simply check: "Can this activation shape explain the error?"
                # Solve: H * w = Residual
                # We use a tiny alpha for stability
                I = cp.eye(n_neurons, dtype=cp.float32)
                # This inverse is tiny (100x100), takes microseconds on GPU
                w_out = cp.linalg.solve(H.T @ H + 1.0*I, H.T @ Residual_gpu)

                # 3. Measure Fit (Energy Reduction)
                pred = H @ w_out
                # Score = How much variance of the residual did we capture?
                # Higher is better.
                error_norm = cp.linalg.norm(Residual_gpu - pred)
                total_norm = cp.linalg.norm(Residual_gpu)
                score = 1.0 - (error_norm / total_norm)

                if score > best_score:
                    best_score = score
                    best_name = name
            except:
                continue

        elapsed = time.time() - start
        print(f" Done in {elapsed:.2f}s.")
        print(f"   [SCANNER] Resonance Detected: '{best_name.upper()}' (Fit Score: {best_score:.4f})")

        return best_name, self.activations[best_name]



# ==========================================
# 2. TITAN INF (The Unified Intelligence) - VALIDATED
# ==========================================
# ==========================================
# 2. TITAN OMEGA (The Universal Intelligence)
# ==========================================
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

class TITAN_INF(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.memory_vault = []
        self.base_model = None
        self.input_dim = 0
        self.n_classes = 0

    def _tournament_selection(self, X, y):
        print(f" > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...")

        # 1. DEFINE THE GLADIATORS
        competitors = [
            ("XGBoost", XGBClassifier(n_estimators=200, use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)),
            ("LightGBM", LGBMClassifier(n_estimators=200, random_state=42, verbosity=-1, n_jobs=-1)),
            ("ExtraTrees", ExtraTreesClassifier(n_estimators=200, n_jobs=-1, random_state=42)),
            ("RandomForest", RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)),
           # ("SVM (RBF)", SVC(probability=True, kernel='rbf', random_state=42)), # Powerful for small/complex data
           # ("KNN (k=5)", KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),       # Good for local geometry
           # ("LogisticReg", LogisticRegression(max_iter=1000, n_jobs=-1)),       # Baseline for linear
           # ("LDA", LinearDiscriminantAnalysis()),
           # ("QDA", QuadraticDiscriminantAnalysis()),                            # Good for Gaussian distributions
           # ("Ridge", RidgeClassifier(random_state=42))                          # Fast linear
        ]

        best_name = "None"
        best_score = -1.0
        best_model = None

        # 2. RAPID BATTLE (3-Fold CV for speed)
        # We use a smaller fold here just to pick the winner quickly
        for name, model in competitors:
            try:
                # Ridge doesn't support predict_proba by default, handle exception or skip
                if name == "Ridge":
                    # Ridge is special, let's skip for probability requirement or calibrate it
                    # For simplicity in this universal script, we skip non-proba models for the base
                    continue

                # Quick check
                score = np.mean(cross_val_score(model, X, y, cv=5, scoring='accuracy', n_jobs=-1))
                print(f"   [SCAR] {name:<12} | Score: {score:.4%}")

                if score > best_score:
                    best_score = score
                    best_name = name
                    best_model = model
            except Exception as e:
                # Some models might fail on specific data types (e.g. QDA on collinear), we ignore them
                pass

        print(f" > [VICTORY] Champion Selected: {best_name.upper()} (Base Acc: {best_score:.4%})")
        return best_model

    def perceive(self, X, y):
        self.input_dim = X.shape[1]
        self.n_classes = len(np.unique(y))

        # 1. SELECT THE PERFECT HOST
        self.base_model = self._tournament_selection(X, y)

        # 2. TRAIN CHAMPION (100% Data)
        print(f" > [TITAN] Assimilating {self.base_model.__class__.__name__}...")
        self.base_model.fit(X, y)

        # 3. GENERATE HONEST RESIDUALS (5-Fold CV)
        print(f" > [TITAN] Generating Holographic Residuals (5-Fold CV)...")
        try:
            cv_probs = cross_val_predict(self.base_model, X, y, cv=5, method='predict_proba', n_jobs=-1)
        except:
            # Fallback if cross_val fails (e.g., extremely small classes)
            print("   [WARNING] CV Failed. Fallback to OOB residuals.")
            self.base_model.fit(X, y)
            cv_probs = self.base_model.predict_proba(X)

        # 4. ENTER THE INFINITE LOOP
        self._fit_evolution(X, y, cv_probs)

    def _fit_evolution(self, X, y, base_probs):
        current_probs = base_probs
        current_preds = np.argmax(current_probs, axis=1)
        best_acc = accuracy_score(y, current_preds)
        current_probs = base_probs
        current_logits = cp.asarray(np.log(current_probs + 1e-9))

        # Initialize Scanner
        self.scanner = RESONANCE_SCANNER()

        X_gpu = cp.asarray(X, dtype=cp.float32)
        current_logits = cp.asarray(np.log(current_probs + 1e-9))
        y_gpu = cp.asarray(y)
        # [UPDATED] The Silencer: Label Smoothing (Prevents Gradient Explosion)
        # Instead of 0.0 and 1.0, we use 0.01 and 0.99.
        # This keeps the "Death Ray" from burning out the weights.
        smooth_factor = 0.01
        Y_target = cp.zeros((len(y), self.n_classes), dtype=cp.float32) + (smooth_factor / self.n_classes)
        for i in range(self.n_classes):
            Y_target[y_gpu==i, i] = 1.0 - smooth_factor

        print(f"   [BASE] Holographic Accuracy: {best_acc:.4%}")

        # --- THE OMEGA LOOP ---
        cycle = 0
        patience = 0
        patience_limit = 30
        max_units = 100000

        while cycle < max_units:
            cycle += 1

            probs = cp.exp(current_logits) / cp.sum(cp.exp(current_logits), axis=1, keepdims=True)
            residual = Y_target - probs

            # 2. GOD'S EYE SCAN (Every 5 cycles to adapt to changing error landscape)
            # We don't need to scan every single cycle, but let's scan occasionally
            # or if patience is getting high.
            if cycle == 1 or cycle % 5 == 0:
                best_af_name, best_af_func = self.scanner.scan(X_gpu, residual)


            # HYPER-MUTATION
            best_unit_cycle = None
            best_acc_cycle = -1
            best_logits_cycle = None

            for _ in range(10):
                n_neurons = np.random.randint(500, 3000)
                alpha = 10 ** np.random.uniform(-1, 4)
                seed = np.random.randint(0, 100000) + cycle

                # PASS THE CHOSEN FUNCTION HERE
                unit = MEMORY_UNIT(n_neurons, self.input_dim, seed, best_af_func)
                unit.fit_residual(X_gpu, residual, alpha=alpha)

                correction = unit.predict_correction(X_gpu)
                lr = 0.1 / (1 + (cycle * 0.01))

                temp_logits = current_logits + (correction * lr)
                temp_preds = cp.asnumpy(cp.argmax(temp_logits, axis=1))
                temp_acc = accuracy_score(y, temp_preds)

                if temp_acc > best_acc_cycle:
                    best_acc_cycle = temp_acc
                    best_unit_cycle = unit
                    best_logits_cycle = temp_logits

            if best_acc_cycle > best_acc:
                self.memory_vault.append(best_unit_cycle)
                current_logits = best_logits_cycle
                best_acc = best_acc_cycle
                patience = 0
                print(f"   [EVOLVE] Cycle {cycle}: IMPROVEMENT -> Acc: {best_acc:.4%}")
            else:
                patience += 1
                if cycle % 5 == 0:
                    print(f"   [FIGHTING] Cycle {cycle}: Stagnant. (Patience {patience}/{patience_limit})")

                if patience >= patience_limit:
                    print(f"   [STOP] Evolution halted. No improvement for {patience_limit} cycles.")
                    break

            if best_acc >= 0.9999:
                print("   [GOD MODE] Perfect Accuracy Achieved. Stopping.")
                break

        print(f"   [TITAN] Evolution Complete. Total Memory Units: {len(self.memory_vault)}. Final Training Acc: {best_acc:.4%}")

    def predict(self, X):
        base_probs = self.base_model.predict_proba(X)
        current_logits = cp.asarray(np.log(base_probs + 1e-9))
        X_gpu = cp.asarray(X, dtype=cp.float32)

        cycle = 0
        for unit in self.memory_vault:
            cycle += 1
            correction = unit.predict_correction(X_gpu)
            lr = 0.1 / (1 + (cycle * 0.01))
            current_logits += (correction * lr)

        return cp.asnumpy(cp.argmax(current_logits, axis=1))

# ==========================================
# 3. THE ARENA ENGINE
# ==========================================
# ==========================================
# 3. THE ARENA ENGINE (UPDATED: VERBOSE LOADER)
# ==========================================
def TITAN_VS_GOLIATHS(data_id, name):
    print(f"\n{'='*100}")
    print(f" >>> PARADIGM SHIFT PROTOCOL: {name.upper()} (ID: {data_id})")
    print(f" >>> OBJECTIVE: Demonstrate Superiority of Infinite Memory over Static Models")
    print(f"{'='*100}")

    # 1. SUMMON DATA (With Status Updates)
    try:
        print(f" > [SYSTEM] Establishing Neural Link to OpenML (ID: {data_id})...")
        start_load = time.time()

        # We explicitly request 'pandas' parser for speed.
        # If this fails, install pyarrow: pip install pyarrow
        data = fetch_openml(data_id=data_id, as_frame=True, parser='pandas')

        print(f" > [SYSTEM] Data Downloaded in {time.time() - start_load:.2f}s. Parsing Structure...")

        X_raw = data.data
        y_raw = data.target

        print(f" > [SYSTEM] Dimensions Detected: {X_raw.shape}. Normalizing entropy...")

        if X_raw.isnull().values.any():
            print("   > [WARNING] Null values detected. Filling with void (0).")
            X_raw = X_raw.fillna(0)

        le = LabelEncoder()
        y = le.fit_transform(y_raw)

        # Handle Categorical Columns by forcing float conversion if possible, or encoding
        # This prevents the "silent hang" when scalars try to eat strings
        X = X_raw.values.astype(np.float32) if hasattr(X_raw, 'values') else X_raw.astype(np.float32)

        print(f" > [SYSTEM] Scaling Geometries (Standard + Quantile)...")
        scaler_std = StandardScaler()
        X_std = scaler_std.fit_transform(X)

        scaler_qt = QuantileTransformer(output_distribution='normal', random_state=42, n_quantiles=min(1000, len(X)))
        X_qt = scaler_qt.fit_transform(X)

        X_train_std, X_test_std, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state=42)
        X_train_qt, X_test_qt, _, _ = train_test_split(X_qt, y, test_size=0.2, random_state=42)

        print(f" > [SYSTEM] Data Ready. Entering The Arena.")

    except Exception as e:
        print(f"\n [FATAL ERROR] Data Summon Failed: {e}")
        print(" [HINT] If this is a connection error, check internet. If 'parser' error, pip install pyarrow.")
        return

    results = []

    # 2. STATIC MODEL BENCHMARKS
    print("\n >>> PHASE 1: STATIC MODEL EVALUATION (THE OLD GUARD)")
    print(f" {'-'*80}")
    competitors = [
        ("XGBoost", XGBClassifier(n_estimators=200, use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)),
        ("LightGBM", LGBMClassifier(n_estimators=200, random_state=42, verbosity=-1, n_jobs=-1)),
        ("ExtraTrees", ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)),
        ("RandomForest", RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)),
    ]

    for model_name, model in competitors:
        start_time = time.time()
        try:
            model.fit(X_train_std, y_train)
            preds = model.predict(X_test_std)
            acc = accuracy_score(y_test, preds)
            elapsed = time.time() - start_time
            results.append({"Model": model_name, "Accuracy": acc, "Time": elapsed})
            print(f"  > [COMPLETED] {model_name:<16} | Accuracy: {acc:.4%} | Time: {elapsed:.2f}s")
        except Exception as e:
            print(f"  > [FAILED]    {model_name:<16} | Error: {e}")

    # 3. TITAN EVOLUTION
    print("\n >>> PHASE 2: TITAN INF AWAKENING (THE NEW PARADIGM)")
    start_time = time.time()

    titan = TITAN_INF()
    # We pass ONLY Training data. The model handles the internal split.
    titan.perceive(X_train_qt, y_train)

    # We predict on Test data WITHOUT passing y_test.
    titan_preds = titan.predict(X_test_qt)
    titan_acc = accuracy_score(y_test, titan_preds)

    elapsed = time.time() - start_time
    results.append({"Model": "TITAN INF (Ours)", "Accuracy": titan_acc, "Time": elapsed})

    # 4. FINAL REPORT
    df = pd.DataFrame(results).sort_values(by="Accuracy", ascending=False)

    print(f"\n{'='*100}")
    print(f" >>> FINAL INTELLIGENCE REPORT: {name}")
    print(f"{'='*100}")
    print(f"{'RANK':<6} | {'MODEL ARCHITECTURE':<25} | {'ACCURACY':<12} | {'ERROR RATE':<12} | {'TIME (s)':<10}")
    print(f"{'-'*100}")

    rank = 1
    for row in df.itertuples():
        error_rate = 1.0 - row.Accuracy
        model_name = row.Model
        if "TITAN" in model_name: model_name = f"** {model_name} **" # Highlight Winner

        print(f" #{rank:<5} | {model_name:<25} | {row.Accuracy:.4%}     | {error_rate:.4%}       | {row.Time:.4f}")
        rank += 1
    print(f"{'='*100}\n")

# --- EXECUTE PROTOCOL ---
if __name__ == "__main__":
    print('Active')

>>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.
Active


In [ ]:
TITAN_VS_GOLIATHS(1494, "QSAR Biodegradation")


 >>> PARADIGM SHIFT PROTOCOL: QSAR BIODEGRADATION (ID: 1494)
 >>> OBJECTIVE: Demonstrate Superiority of Infinite Memory over Static Models
 > [SYSTEM] Establishing Neural Link to OpenML (ID: 1494)...
 > [SYSTEM] Data Downloaded in 0.02s. Parsing Structure...
 > [SYSTEM] Dimensions Detected: (1055, 41). Normalizing entropy...
 > [SYSTEM] Scaling Geometries (Standard + Quantile)...
 > [SYSTEM] Data Ready. Entering The Arena.

 >>> PHASE 1: STATIC MODEL EVALUATION (THE OLD GUARD)
 --------------------------------------------------------------------------------
  > [COMPLETED] XGBoost          | Accuracy: 87.2038% | Time: 0.18s
  > [COMPLETED] LightGBM         | Accuracy: 87.2038% | Time: 0.26s
  > [COMPLETED] ExtraTrees       | Accuracy: 87.6777% | Time: 0.40s
  > [COMPLETED] RandomForest     | Accuracy: 88.6256% | Time: 0.57s

 >>> PHASE 2: TITAN INF AWAKENING (THE NEW PARADIGM)
 > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...
   [SCAR] XGBoost      | Score: 85.0690%
   [SCA

In [ ]:
TITAN_VS_GOLIATHS(4534, "Phishing Websites")


 >>> PARADIGM SHIFT PROTOCOL: PHISHING WEBSITES (ID: 4534)
 >>> OBJECTIVE: Demonstrate Superiority of Infinite Memory over Static Models
 > [SYSTEM] Establishing Neural Link to OpenML (ID: 4534)...
 > [SYSTEM] Data Downloaded in 0.05s. Parsing Structure...
 > [SYSTEM] Dimensions Detected: (11055, 30). Normalizing entropy...
 > [SYSTEM] Scaling Geometries (Standard + Quantile)...
 > [SYSTEM] Data Ready. Entering The Arena.

 >>> PHASE 1: STATIC MODEL EVALUATION (THE OLD GUARD)
 --------------------------------------------------------------------------------
  > [COMPLETED] XGBoost          | Accuracy: 97.2863% | Time: 0.23s
  > [COMPLETED] LightGBM         | Accuracy: 96.9697% | Time: 0.29s
  > [COMPLETED] ExtraTrees       | Accuracy: 96.7888% | Time: 1.00s
  > [COMPLETED] RandomForest     | Accuracy: 96.7888% | Time: 0.86s

 >>> PHASE 2: TITAN INF AWAKENING (THE NEW PARADIGM)
 > [TITAN] Creating Oracle Holdout Layer (80/20 Split)...
 > [G.O.D. PROTOCOL] Initiating Host Selection Tourn

In [ ]:
TITAN_VS_GOLIATHS(1479, "Hill Valley")


 >>> PARADIGM SHIFT PROTOCOL: HILL VALLEY (ID: 1479)
 >>> OBJECTIVE: Demonstrate Superiority of Infinite Memory over Static Models
 > [SYSTEM] Establishing Neural Link to OpenML (ID: 1479)...
 > [SYSTEM] Data Downloaded in 0.04s. Parsing Structure...
 > [SYSTEM] Dimensions Detected: (1212, 100). Normalizing entropy...
 > [SYSTEM] Scaling Geometries (Standard + Quantile)...
 > [SYSTEM] Data Ready. Entering The Arena.

 >>> PHASE 1: STATIC MODEL EVALUATION (THE OLD GUARD)
 --------------------------------------------------------------------------------
  > [COMPLETED] XGBoost          | Accuracy: 53.0864% | Time: 1.14s
  > [COMPLETED] LightGBM         | Accuracy: 59.2593% | Time: 1.35s
  > [COMPLETED] ExtraTrees       | Accuracy: 58.0247% | Time: 0.55s
  > [COMPLETED] RandomForest     | Accuracy: 55.9671% | Time: 2.16s

 >>> PHASE 2: TITAN INF AWAKENING (THE NEW PARADIGM)
 > [TITAN] Creating Oracle Holdout Layer (80/20 Split)...
 > [G.O.D. PROTOCOL] Initiating Host Selection Tournament.

In [ ]:
TITAN_VS_GOLIATHS(40499, "Texture Analysis")


 >>> PARADIGM SHIFT PROTOCOL: TEXTURE ANALYSIS (ID: 40499)
 >>> OBJECTIVE: Demonstrate Superiority of Infinite Memory over Static Models

 >>> PHASE 1: STATIC MODEL EVALUATION (THE OLD GUARD)
 --------------------------------------------------------------------------------

 >>> PHASE 2: TITAN INF AWAKENING (THE NEW PARADIGM)
 > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...
   [SCAR] XGBoost      | Score: 97.9091%
   [SCAR] LightGBM     | Score: 98.3182%
   [SCAR] ExtraTrees   | Score: 98.2273%
   [SCAR] RandomForest | Score: 97.5455%
   [SCAR] SVM (RBF)    | Score: 99.0682%
   [SCAR] KNN (k=5)    | Score: 97.9545%
   [SCAR] LogisticReg  | Score: 98.9773%
   [SCAR] LDA          | Score: 98.8182%
   [SCAR] QDA          | Score: 13.6136%
 > [VICTORY] Champion Selected: SVM (RBF) (Base Acc: 99.0682%)
 > [TITAN] Assimilating SVC...
 > [TITAN] Generating Holographic Residuals (5-Fold CV)...
   [BASE] Holographic Accuracy: 99.0000%
   [SCANNER] Testing 95 Quantum Activation Func

In [ ]:
TITAN_VS_GOLIATHS(1471, "EEG Eye State")


 >>> PARADIGM SHIFT PROTOCOL: EEG EYE STATE (ID: 1471)
 >>> OBJECTIVE: Demonstrate Superiority of Infinite Memory over Static Models
 > [SYSTEM] Establishing Neural Link to OpenML (ID: 1471)...
 > [SYSTEM] Data Downloaded in 2.96s. Parsing Structure...
 > [SYSTEM] Dimensions Detected: (14980, 14). Normalizing entropy...
 > [SYSTEM] Scaling Geometries (Standard + Quantile)...
 > [SYSTEM] Data Ready. Entering The Arena.

 >>> PHASE 1: STATIC MODEL EVALUATION (THE OLD GUARD)
 --------------------------------------------------------------------------------
  > [COMPLETED] XGBoost          | Accuracy: 94.1923% | Time: 0.44s
  > [COMPLETED] LightGBM         | Accuracy: 92.4566% | Time: 0.44s
  > [COMPLETED] ExtraTrees       | Accuracy: 94.6929% | Time: 1.95s
  > [COMPLETED] RandomForest     | Accuracy: 92.3565% | Time: 3.96s

 >>> PHASE 2: TITAN INF AWAKENING (THE NEW PARADIGM)
 > [TITAN] Summoning The Council (Layer 1)...
   [LAYER 1] Training RF...
   [LAYER 1] Training ET...
   [LAYER 1]

In [ ]:
TITAN_VS_GOLIATHS(1489, "Phoneme Acoustics")


 >>> PARADIGM SHIFT PROTOCOL: PHONEME ACOUSTICS (ID: 1489)
 >>> OBJECTIVE: Demonstrate Superiority of Infinite Memory over Static Models
 > [SYSTEM] Establishing Neural Link to OpenML (ID: 1489)...
 > [SYSTEM] Data Downloaded in 0.02s. Parsing Structure...
 > [SYSTEM] Dimensions Detected: (5404, 5). Normalizing entropy...
 > [SYSTEM] Scaling Geometries (Standard + Quantile)...
 > [SYSTEM] Data Ready. Entering The Arena.

 >>> PHASE 1: STATIC MODEL EVALUATION (THE OLD GUARD)
 --------------------------------------------------------------------------------
  > [COMPLETED] XGBoost          | Accuracy: 90.9343% | Time: 0.22s
  > [COMPLETED] LightGBM         | Accuracy: 90.0093% | Time: 0.17s
  > [COMPLETED] ExtraTrees       | Accuracy: 91.0268% | Time: 0.61s
  > [COMPLETED] RandomForest     | Accuracy: 90.9343% | Time: 1.57s

 >>> PHASE 2: TITAN INF AWAKENING (THE NEW PARADIGM)
 > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...
   [SCAR] XGBoost      | Score: 89.3361%
   [SCAR] 

In [ ]:
#TITAN_VS_GOLIATHS(1120, "Magic Gamma Telescope")

In [ ]:
#TITAN_VS_GOLIATHS(151, "Electricity Grid Stability")

In [ ]:
import numpy as np
import sys
import warnings
import time
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# --- COMPETITORS ---
try:
    from xgboost import XGBClassifier
    from lightgbm import LGBMClassifier
except ImportError:
    print(">>> [WARNING] XGBoost or LightGBM not installed. Benchmarks will be limited.")

# --- GPU ACTIVATION ---
try:
    import cupy as cp
    print(">>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.")
    cp.random.seed(42)
    GPU_AVAILABLE = True
except ImportError:
    print(">>> [FATAL] GPU REQUIRED FOR TITAN.")
    sys.exit()

warnings.filterwarnings("ignore")

# ==========================================
# 1. THE MEMORY UNIT (The Synapse)
# ==========================================
class MEMORY_UNIT:
    def __init__(self, n_neurons, input_dim, seed, activation_func):
        cp.random.seed(seed)
        self.n_neurons = n_neurons
        self.activation = activation_func
        self.scaling_factor = 1.0

        self.W_in = cp.random.uniform(-1.0, 1.0, (n_neurons, input_dim)).astype(cp.float32)
        self.W_cortex = cp.random.normal(0, 1.0 / np.sqrt(n_neurons), (n_neurons, n_neurons)).astype(cp.float32)

        # Spectral stabilizer
        try:
            v = cp.random.randn(n_neurons)
            for _ in range(5):
                v = self.W_cortex @ v
                v = v / cp.linalg.norm(v)
            max_eig = cp.linalg.norm(self.W_cortex @ v)
            if max_eig > 0: self.W_cortex /= max_eig
        except: pass
        self.readout = None

    def process(self, X_gpu):
        state = cp.zeros((X_gpu.shape[0], self.n_neurons), dtype=cp.float32)
        for _ in range(5):
            pre = (X_gpu @ self.W_in.T) + (state @ self.W_cortex.T)
            state = self.activation(pre)
        return state

    def fit_residual(self, X_gpu, Residual_gpu, alpha=10.0):
        H = self.process(X_gpu)
        I = cp.eye(self.n_neurons, dtype=cp.float32)
        self.readout = cp.linalg.solve(H.T @ H + alpha*I, H.T @ Residual_gpu)

    def predict_correction(self, X_gpu):
        H = self.process(X_gpu)
        return H @ self.readout


# ==========================================
# 2. THE RESONANCE SCANNER (God's Eye)
# ==========================================
class RESONANCE_SCANNER:
    def __init__(self):
        self.activations = {}
        self._generate_100_functions()

    def _generate_100_functions(self):
        # 1. Standard Activations
        self.activations['tanh'] = lambda x: cp.tanh(x)
        self.activations['relu'] = lambda x: cp.maximum(0, x)
        self.activations['sigmoid'] = lambda x: 1 / (1 + cp.exp(-x))
        self.activations['swish'] = lambda x: x * (1 / (1 + cp.exp(-x)))
        self.activations['mish'] = lambda x: x * cp.tanh(cp.log(1 + cp.exp(x)))

        # 2. Harmonic Series (The "Siren" Frequencies)
        for freq in [0.5, 1.0, 1.5, 2.0, 3.0, 5.0, 8.0, 13.0, 21.0, 34.0, 55.0]:
            self.activations[f'sin_omega_{freq}'] = lambda x, f=freq: cp.sin(f * x)
            self.activations[f'cos_omega_{freq}'] = lambda x, f=freq: cp.cos(f * x)

        # 3. Gaussian Kernels
        for sigma in [0.1, 0.5, 1.0, 2.0, 5.0]:
            self.activations[f'gaussian_{sigma}'] = lambda x, s=sigma: cp.exp(-cp.square(x) / (2 * s**2))

        # 4. Hybrid & Exotic
        self.activations['sinc'] = lambda x: cp.sin(x) / (x + 1e-6)
        self.activations['morlet'] = lambda x: cp.cos(5*x) * cp.exp(-x**2/2)
        self.activations['x_sin_x'] = lambda x: x * cp.sin(x)
        self.activations['sin_tanh'] = lambda x: cp.sin(cp.tanh(x))

        # Fine-tuned bands
        for i in range(1, 40):
            f = 0.1 * i
            self.activations[f'fine_sin_{f:.1f}'] = lambda x, freq=f: cp.sin(freq * x)

    def scan(self, X_gpu, Residual_gpu, n_neurons=100):
        best_name = "tanh"
        best_score = -np.inf
        cp.random.seed(42)
        W_probe = cp.random.uniform(-1.0, 1.0, (n_neurons, X_gpu.shape[1])).astype(cp.float32)
        Projected_X = X_gpu @ W_probe.T

        print(f"   [SCANNER] Testing {len(self.activations)} Quantum Activation Functions...", end="")
        start = time.time()

        for name, func in self.activations.items():
            try:
                H = func(Projected_X)
                I = cp.eye(n_neurons, dtype=cp.float32)
                w_out = cp.linalg.solve(H.T @ H + 1.0*I, H.T @ Residual_gpu)
                pred = H @ w_out
                error_norm = cp.linalg.norm(Residual_gpu - pred)
                total_norm = cp.linalg.norm(Residual_gpu)
                score = 1.0 - (error_norm / total_norm)

                if score > best_score:
                    best_score = score
                    best_name = name
            except: continue

        elapsed = time.time() - start
        print(f" Done in {elapsed:.2f}s.")
        print(f"   [SCANNER] Resonance Detected: '{best_name.upper()}' (Fit Score: {best_score:.4f})")
        return best_name, self.activations[best_name]


# ==========================================
# 3. TITAN GLASS (The Edge Case Sniper)
# ==========================================
class TITAN_GLASS:
    def __init__(self, input_dim, n_classes, mirror_dim=4096):
        self.input_dim = input_dim
        self.n_classes = n_classes
        self.mirror_dim = mirror_dim
        self.W_mirror = None
        self.readout = None

    def fit(self, X_gpu, current_probs_gpu, Residual_gpu, y_gpu):
        """
        Learns to correct ONLY the samples where the Base + Memory Vault failed.
        """
        print(f" > [TITAN GLASS] Focusing Infinite Mirrors on Edge Cases (Dim: {self.mirror_dim})...")

        # 1. IDENTIFY THE MISTAKES
        # We look at where the current 'Best' prediction disagrees with Reality
        current_preds = cp.argmax(current_probs_gpu, axis=1)
        mistakes_mask = (current_preds != y_gpu)
        n_mistakes = cp.sum(mistakes_mask).item()

        print(f"   [GLASS] Detected {n_mistakes} Stubborn Edge Cases in Training Data.")

        if n_mistakes == 0:
            print(f"   [GLASS] Zero errors detected. The Reflection is unnecessary.")
            return

        # 2. GENERATE FOCUS WEIGHTS
        # We tell the math solver: "Paying attention to a MISTAKE is 50x more important"
        n_samples = X_gpu.shape[0]
        weights = cp.ones(n_samples, dtype=cp.float32)
        weights[mistakes_mask] = 50.0

        # Square root of weights for the Linear Algebra trick
        sqrt_W = cp.sqrt(weights)[:, None] # Shape (N, 1)

        # 3. PROJECT INTO INFINITE DIMENSIONS
        cp.random.seed(1337)
        total_in_dim = self.input_dim + self.n_classes
        self.W_mirror = cp.random.normal(0, 1.0 / np.sqrt(self.mirror_dim),
                                       (self.mirror_dim, total_in_dim)).astype(cp.float32)

        Combined_State = cp.hstack([X_gpu, current_probs_gpu])

        # 4. APPLY SIREN ACTIVATION (High Frequency)
        Projection = Combined_State @ self.W_mirror.T
        H = cp.sin(Projection)

        # 5. WEIGHTED RIDGE REGRESSION
        H_weighted = H * sqrt_W
        Residual_weighted = Residual_gpu * sqrt_W

        alpha = 10.0
        I = cp.eye(self.mirror_dim, dtype=cp.float32)

        try:
            self.readout = cp.linalg.solve(H_weighted.T @ H_weighted + alpha*I, H_weighted.T @ Residual_weighted)
            print(f"   [TITAN GLASS] Mirrors Aligned. Focus applied to {n_mistakes} samples.")
        except:
            print("   [TITAN GLASS] Singularity. Bypassing.")
            self.readout = None

    def reflect(self, X_gpu, current_probs_gpu):
        if self.readout is None:
            return 0.0

        Combined_State = cp.hstack([X_gpu, current_probs_gpu])
        Projection = Combined_State @ self.W_mirror.T
        H = cp.sin(Projection)
        return H @ self.readout


# ==========================================
# 4. TITAN INF (The Unified Intelligence)
# ==========================================
class TITAN_INF(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.memory_vault = []
        self.base_model = None
        self.glass = None
        self.input_dim = 0
        self.n_classes = 0
        self.scanner = None

    def _compress_vault(self, X_gpu, batch_size=100):
        if len(self.memory_vault) < batch_size * 2: return
        print(f"   [TITAN] Compressing Memory Vault (Distilling {batch_size} units)...")
        units_to_compress = self.memory_vault[:batch_size]
        remaining_units = self.memory_vault[batch_size:]
        total_correction = cp.zeros((X_gpu.shape[0], self.n_classes), dtype=cp.float32)
        for unit in units_to_compress:
            pred = unit.predict_correction(X_gpu)
            total_correction += pred * unit.scaling_factor
        super_unit = MEMORY_UNIT(n_neurons=3000, input_dim=self.input_dim, seed=42, activation_func=cp.tanh)
        super_unit.fit_residual(X_gpu, total_correction, alpha=1.0)
        super_unit.scaling_factor = 1.0
        self.memory_vault = [super_unit] + remaining_units
        print(f"   [TITAN] Compression Complete. Vault Size: {len(self.memory_vault)}")

    def _tournament_selection(self, X, y):
        print(f" > [G.O.D. PROTOCOL] Initiating Host Selection Tournament...")
        competitors = [
            ("XGBoost", XGBClassifier(n_estimators=200, use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)),
            ("LightGBM", LGBMClassifier(n_estimators=200, random_state=42, verbosity=-1, n_jobs=-1)),
            ("ExtraTrees", ExtraTreesClassifier(n_estimators=200, n_jobs=-1, random_state=42)),
            ("RandomForest", RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)),
            ("SVM (RBF)", SVC(probability=True, kernel='rbf', random_state=42)),
            ("KNN (k=5)", KNeighborsClassifier(n_neighbors=5, n_jobs=-1)),
            ("LogisticReg", LogisticRegression(max_iter=1000, n_jobs=-1)),
            ("LDA", LinearDiscriminantAnalysis()),
            ("QDA", QuadraticDiscriminantAnalysis()),
        ]
        best_name, best_score, best_model = "None", -1.0, None
        for name, model in competitors:
            try:
                score = np.mean(cross_val_score(model, X, y, cv=5, scoring='accuracy', n_jobs=-1))
                print(f"   [SCAR] {name:<12} | Score: {score:.4%}")
                if score > best_score:
                    best_score = score
                    best_name = name
                    best_model = model
            except: pass
        print(f" > [VICTORY] Champion Selected: {best_name.upper()} (Base Acc: {best_score:.4%})")
        return best_model

    def perceive(self, X, y):
        self.input_dim = X.shape[1]
        self.n_classes = len(np.unique(y))

        # 1. Host Selection
        self.base_model = self._tournament_selection(X, y)
        print(f" > [TITAN] Assimilating {self.base_model.__class__.__name__}...")
        self.base_model.fit(X, y)

        # 2. Residual Generation
        print(f" > [TITAN] Generating Holographic Residuals (5-Fold CV)...")
        try:
            cv_probs = cross_val_predict(self.base_model, X, y, cv=5, method='predict_proba', n_jobs=-1)
        except:
            print("   [WARNING] CV Failed. Fallback to OOB residuals.")
            self.base_model.fit(X, y)
            cv_probs = self.base_model.predict_proba(X)

        # 3. Evolution Loop
        self._fit_evolution(X, y, cv_probs)

    def _fit_evolution(self, X, y, base_probs):
        X_gpu = cp.asarray(X, dtype=cp.float32)
        y_gpu = cp.asarray(y)
        current_probs = base_probs
        current_logits = cp.asarray(np.log(current_probs + 1e-9))

        # Label Smoothing Target
        smooth_factor = 0.01
        Y_target = cp.zeros((len(y), self.n_classes), dtype=cp.float32) + (smooth_factor / self.n_classes)
        for i in range(self.n_classes):
            Y_target[y_gpu==i, i] = 1.0 - smooth_factor

        best_acc = accuracy_score(y, np.argmax(base_probs, axis=1))
        print(f"   [BASE] Holographic Accuracy: {best_acc:.4%}")

        self.scanner = RESONANCE_SCANNER()
        cycle = 0
        patience = 0
        patience_limit = 30
        max_units = 100000

        # --- THE OMEGA LOOP ---
        while cycle < max_units:
            cycle += 1
            probs = cp.exp(current_logits) / cp.sum(cp.exp(current_logits), axis=1, keepdims=True)
            residual = Y_target - probs

            if cycle == 1 or cycle % 5 == 0:
                best_af_name, best_af_func = self.scanner.scan(X_gpu, residual)

            # HYPER-MUTATION
            best_unit_cycle = None
            best_acc_cycle = -1
            best_logits_cycle = None

            for _ in range(10): # 10 Mutations per cycle
                n_neurons = np.random.randint(500, 3000)
                alpha = 10 ** np.random.uniform(-1, 4)
                seed = np.random.randint(0, 100000) + cycle

                unit = MEMORY_UNIT(n_neurons, self.input_dim, seed, best_af_func)
                unit.fit_residual(X_gpu, residual, alpha=alpha)

                correction = unit.predict_correction(X_gpu)
                lr = 0.1 / (1 + (cycle * 0.01))

                temp_logits = current_logits + (correction * lr)
                temp_preds = cp.asnumpy(cp.argmax(temp_logits, axis=1))
                temp_acc = accuracy_score(y, temp_preds)

                if temp_acc > best_acc_cycle:
                    best_acc_cycle = temp_acc
                    best_unit_cycle = unit
                    best_logits_cycle = temp_logits

            if best_acc_cycle > best_acc:
                best_unit_cycle.scaling_factor = 0.1 / (1 + (cycle * 0.01))
                self.memory_vault.append(best_unit_cycle)
                current_logits = best_logits_cycle
                best_acc = best_acc_cycle
                patience = 0
                print(f"   [EVOLVE] Cycle {cycle}: IMPROVEMENT -> Acc: {best_acc:.4%}")

                if len(self.memory_vault) % 100 == 0:
                    self._compress_vault(X_gpu, batch_size=50)
            else:
                patience += 1
                if cycle % 5 == 0:
                    print(f"   [FIGHTING] Cycle {cycle}: Stagnant. (Patience {patience}/{patience_limit})")

                if patience >= patience_limit:
                    print(f"   [STOP] Evolution halted. No improvement for {patience_limit} cycles.")
                    break

            if best_acc >= 0.9999:
                print("   [GOD MODE] Perfect Accuracy Achieved. Stopping.")
                break

        # --- PHASE 3: TITAN GLASS ACTIVATION ---
        # Recalculate Final State for the Mirror
        final_probs = cp.exp(current_logits) / cp.sum(cp.exp(current_logits), axis=1, keepdims=True)
        final_residual = Y_target - final_probs

        self.glass = TITAN_GLASS(self.input_dim, self.n_classes, mirror_dim=4096)
        # *** FIX: PASSING y_gpu HERE ***
        self.glass.fit(X_gpu, final_probs, final_residual, y_gpu)

        print(f"   [TITAN] Evolution Complete. Total Memory Units: {len(self.memory_vault)}. Final Training Acc: {best_acc:.4%}")

    def predict(self, X):
        # 1. Base Prediction
        base_probs = self.base_model.predict_proba(X)
        current_logits = cp.asarray(np.log(base_probs + 1e-9))
        X_gpu = cp.asarray(X, dtype=cp.float32)

        # 2. Memory Vault Correction
        for unit in self.memory_vault:
            correction = unit.predict_correction(X_gpu)
            current_logits += (correction * unit.scaling_factor)

        # 3. TITAN GLASS Correction (The Final Reflection)
        current_probs = cp.exp(current_logits) / cp.sum(cp.exp(current_logits), axis=1, keepdims=True)

        if self.glass is not None:
            reflection = self.glass.reflect(X_gpu, current_probs)
            current_logits += reflection

        return cp.asnumpy(cp.argmax(current_logits, axis=1))


# ==========================================
# 5. THE ARENA ENGINE
# ==========================================
def TITAN_VS_GOLIATHS(data_id, name):
    print(f"\n{'='*100}")
    print(f" >>> PARADIGM SHIFT PROTOCOL: {name.upper()} (ID: {data_id})")
    print(f" >>> OBJECTIVE: Prove Superiority of Infinite Reflection & Memory")
    print(f"{'='*100}")

    # 1. SUMMON DATA
    try:
        data = fetch_openml(data_id=data_id, as_frame=True, parser='auto')
        X_raw = data.data
        y_raw = data.target
        if X_raw.isnull().values.any(): X_raw = X_raw.fillna(0)
        le = LabelEncoder()
        y = le.fit_transform(y_raw)
        X = X_raw.values.astype(np.float32) if hasattr(X_raw, 'values') else X_raw.astype(np.float32)

        scaler_std = StandardScaler()
        X_std = scaler_std.fit_transform(X)

        scaler_qt = QuantileTransformer(output_distribution='normal', random_state=42, n_quantiles=min(1000, len(X)))
        X_qt = scaler_qt.fit_transform(X)

        X_train_std, X_test_std, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state=42)
        X_train_qt, X_test_qt, _, _ = train_test_split(X_qt, y, test_size=0.2, random_state=42)

    except Exception as e:
        print(f" [ERROR] Data Summon Failed: {e}")
        return

    results = []

    # 2. STATIC MODEL BENCHMARKS
    print("\n >>> PHASE 1: STATIC MODEL EVALUATION")
    print(f" {'-'*80}")
    competitors = [
      # ("RandomForest", RandomForestClassifier(n_estimators=100, random_state=42)),
       #("ExtraTrees", ExtraTreesClassifier(n_estimators=100, random_state=42)),
      # ("XGBoost", XGBClassifier(n_estimators=100, eval_metric='logloss', random_state=42))
    ]

    for model_name, model in competitors:
        start_time = time.time()
        try:
            model.fit(X_train_std, y_train)
            preds = model.predict(X_test_std)
            acc = accuracy_score(y_test, preds)
            elapsed = time.time() - start_time
            results.append({"Model": model_name, "Accuracy": acc, "Time": elapsed})
            print(f"  > {model_name:<16} | Acc: {acc:.4%} | Time: {elapsed:.2f}s")
        except Exception as e:
            print(f"  > {model_name:<16} | Error: {e}")

    # 3. TITAN EVOLUTION
    print("\n >>> PHASE 2: TITAN INF (EVOLUTIONARY MEMORY)")
    start_time = time.time()

    titan = TITAN_INF()
    titan.perceive(X_train_qt, y_train) # Train

    print("\n >>> PHASE 3: TITAN GLASS (INFINITE REFLECTION)")
    # Prediction (Glass is applied automatically in predict())
    titan_preds = titan.predict(X_test_qt)
    titan_acc = accuracy_score(y_test, titan_preds)

    elapsed = time.time() - start_time
    results.append({"Model": "TITAN INF", "Accuracy": titan_acc, "Time": elapsed})

    # 4. FINAL REPORT
    df = pd.DataFrame(results).sort_values(by="Accuracy", ascending=False)

    print(f"\n{'='*100}")
    print(f" >>> FINAL INTELLIGENCE REPORT: {name}")
    print(f"{'='*100}")
    print(f"{'RANK':<6} | {'MODEL ARCHITECTURE':<25} | {'ACCURACY':<12} | {'ERROR RATE':<12}")
    print(f"{'-'*100}")

    rank = 1
    for row in df.itertuples():
        error_rate = 1.0 - row.Accuracy
        model_name = row.Model
        if "TITAN" in model_name: model_name = f"** {model_name} **"

        print(f" #{rank:<5} | {model_name:<25} | {row.Accuracy:.4%}     | {error_rate:.4%}")
        rank += 1
    print(f"{'='*100}\n")

# --- EXECUTE PROTOCOL ---
if __name__ == "__main__":
    print('Active')


>>> [TITAN PROTOCOL] T4 GPU DETECTED. INFINITE MEMORY ENGINE ONLINE.
Active


In [ ]:
#TITAN_VS_GOLIATHS(1479, "Hill Valley")

In [ ]:
#TITAN_VS_GOLIATHS(1120, "Magic Gamma Telescope")

In [ ]:
#TITAN_VS_GOLIATHS(1494, "QSAR Biodegradation")

In [ ]:
#TITAN_VS_GOLIATHS(4534, "Phishing Websites")